In [1]:
import numpy as np
import pandas as pd
from glob import glob
import os

In [16]:
snr_cat = pd.read_pickle('../dataframes/high_snr_df.pkl')

In [17]:
idx = 1
for i in snr_cat.index:
    slrm_text= ["#!/bin/bash\n#SBATCH -J bfit-{0}\n#SBATCH -p background\n#SBATCH --time=8:00:00\n#SBATCH --mem-per-cpu=32000\n#SBATCH -o bfit{0}-%j.out\n\
#SBATCH -e bfit{0}-%j.err\n\nchmod a+x bestfit.py\n\n\
python bestfit.py {1} {2} {3} &\n\
\nwait\n\nexit 0".format(idx, snr_cat.field[i],snr_cat.id[i], np.round(snr_cat.z[i],4))]
    np.savetxt('../cluster_scripts/bestfit_{0}.slrm'.format(idx), slrm_text, fmt='%s')   
    idx +=1


In [12]:
print(slrm_text[0])

#!/bin/bash
#SBATCH -J bfit-61
#SBATCH -p background
#SBATCH --time=2:00:00
#SBATCH --mem-per-cpu=16000
#SBATCH -o bfit61-%j.out
#SBATCH -e bfit61-%j.err

chmod a+x bestfit.py

python bestfit.py GSD 35774 1.2533 &

wait

exit 0


In [5]:
flist = glob('../data/bestfits/*bestfit.npy')

field = [os.path.basename(U).split('_')[0] for U in flist]
galaxy = np.array([os.path.basename(U).split('_')[1] for U in flist]).astype(np.int)

In [3]:
from dynesty.utils import quantile as _quantile
from scipy.ndimage import gaussian_filter as norm_kde

def Get_posterior(sample,logwt,logz):
    weight = np.exp(logwt - logz[-1])

    q = [0.5 - 0.5 * 0.999999426697, 0.5 + 0.5 * 0.999999426697]
    span = _quantile(sample.T, q, weights=weight)

    s = 0.02

    bins = int(round(10. / 0.02))
    n, b = np.histogram(sample, bins=bins, weights=weight,
                        range=np.sort(span))
    n = norm_kde(n, 10.)
    x0 = 0.5 * (b[1:] + b[:-1])
    y0 = n
    
    return x0, y0 / np.trapz(y0,x0)

In [8]:
idx = 1
for i in range(len(field)):   
    bfZ, bft, bftau1, bftau2, bftau3, bftau4, bftau5, bftau6, bfz, bfd =np.load('../data/bestfits/{0}_{1}_bestfit.npy'.format(field[i],
                                                                                                                              galaxy[i]))
    
    slrm_text= ["#!/bin/bash\n#SBATCH -J nfit-{0}\n#SBATCH -p background\n#SBATCH --time=16:00:00\n#SBATCH --mem-per-cpu=16000\n#SBATCH -o nfit{0}-%j.out\n\
#SBATCH -e nfit{0}-%j.err\n\nchmod a+x fullfit.py\n\n\
python fullfit.py {1} {2} {3:0.4f}\n\
\nwait\n\nexit 0".format(idx, field[i], galaxy[i], bfz)]
    np.savetxt('../cluster_scripts/nestfit_{0}.slrm'.format(idx), slrm_text, fmt='%s')   
    idx +=1                  
        

In [31]:
tau = ['tab', 'delay']
trim = [0,2500]
trails = [1,2,3,4,5]
idx=1
for i in range(len(tau)):
    for ii in range(len(trim)):
        for iii in range(len(trails)):        
            slrm_text= ["#!/bin/bash\n#SBATCH -J tfit-{0}\n#SBATCH -p background\n#SBATCH --time=16:00:00\n#SBATCH --mem-per-cpu=16000\n#SBATCH -o tfit{0}-%j.out\n\
#SBATCH -e tfit{0}-%j.err\n\nchmod a+x fullfit-test.py\n\n\
python fullfit-test.py {1} {2} {3}\nnwait\n\nexit 0".format(idx, trails[iii], tau[i], trim[ii])]
            np.savetxt('../cluster_scripts/testfit_{0}.slrm'.format(idx), slrm_text, fmt='%s') 
            idx+=1

In [7]:
##### bestfit

idx = 1
for i in range(len(field)):   
    slrm_text= ["#!/bin/bash\n#SBATCH -J bfit-{0}\n#SBATCH -p background\n#SBATCH --time=16:00:00\n#SBATCH --mem-per-cpu=16000\n#SBATCH -o bfit{0}-%j.out\n\
#SBATCH -e bfit{0}-%j.err\n\nchmod a+x bestfit.py\n\n\
python bestfit.py {1} {2}\n\
\nwait\n\nexit 0".format(idx, field[i], galaxy[i])]
    np.savetxt('../cluster_scripts/bestfit_{0}.slrm'.format(idx), slrm_text, fmt='%s')   
    idx +=1  

In [7]:
print(slrm_text[0])

#!/bin/bash
#SBATCH -J nfit-56
#SBATCH -p background
#SBATCH --time=16:00:00
#SBATCH --mem-per-cpu=16000
#SBATCH -o nfit56-%j.out
#SBATCH -e nfit56-%j.err

chmod a+x fullfit.py

python fullfit.py GSD 47691 1.1287

wait

exit 0


In [21]:
##### testfit

idx = 1

for i in range(1,5,1):
    for ii in range(10):
        slrm_text= ["#!/bin/bash\n#SBATCH -J tfit-{0}\n#SBATCH -p background\n#SBATCH -c 8\n\
#SBATCH --time=16:00:00\n#SBATCH --mem=32gb\n#SBATCH -o tfit{0}-%j.out\n#SBATCH -e tfit{0}-%j.err\n\nchmod a+x testfit_{1}.py\n\n\
python testfit_{1}.py {2} {3}\n\
\nwait\n\nexit 0".format(idx, i + 10, ii +1, ii + 7)]
        np.savetxt('../cluster_scripts/testfit_{0}.slrm'.format(idx), slrm_text, fmt='%s')   
        idx +=1  

In [19]:
print(slrm_text[0])

#!/bin/bash
#SBATCH -J tfit-40
#SBATCH -p background
#SBATCH -c 8
#SBATCH --time=16:00:00
#SBATCH --mem=32gb
#SBATCH -o tfit40-%j.out
#SBATCH -e tfit40-%j.err

chmod a+x testfit_14.py

python testfit_14.py 10 16

wait

exit 0


In [13]:
####ALMA Zfit########
gals = [26272, 29257, 38843, 39012, 39364, 40223, 41520, 42113, 42778,
        43007, 44042]

LIMSR = [None, None, None, None, None, [11200,12500], None, [11200,15400], [14000,16500], None, None]

idx = 1

for i in range(len(gals)):
    if LIMSR[i] == None:
        lim1 = 11200
        lim2 = 16500
    else:
        lim1 = LIMSR[i][0]
        lim2 = LIMSR[i][1]
    
    slrm_text= ["#!/bin/bash\n#SBATCH -J afit-{0}\n#SBATCH -p background\n#SBATCH -c 8\n\
#SBATCH --time=16:00:00\n#SBATCH --mem=32gb\n#SBATCH -o afit{0}-%j.out\n#SBATCH -e afit{0}-%j.err\n\nchmod a+x ALMA_Zfit.py\n\n\
python ALMA_Zfit.py {1} {2} {3}\n\
\nwait\n\nexit 0".format(idx, gals[i], lim1, lim2)]
    np.savetxt('../cluster_scripts/ALMA_Zfit_{0}.slrm'.format(idx), slrm_text, fmt='%s')   
    idx +=1  

In [12]:
print(slrm_text[0])

#!/bin/bash
#SBATCH -J afit-11
#SBATCH -p background
#SBATCH -c 8
#SBATCH --time=16:00:00
#SBATCH --mem=32gb
#SBATCH -o afit11-%j.out
#SBATCH -e afit11-%j.err

chmod a+x ALMA_Zfit.py

python ALMA_Zfit.py 44042 11200 16500

wait

exit 0
